In [8]:
import csv
import io
import nltk
import pandas as pd
import numpy as np
import matplotlib
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from google.colab import files
uploaded = files.upload()

In [0]:
x=pd.read_csv('KDDCup99.csv')

In [10]:
# Lists out all the unwanted intruders, only normal is ideal.
intruders=list(x['label'])
set(intruders)

{'back',
 'buffer_overflow',
 'ftp_write',
 'guess_passwd',
 'imap',
 'ipsweep',
 'land',
 'loadmodule',
 'multihop',
 'neptune',
 'nmap',
 'normal',
 'perl',
 'phf',
 'pod',
 'portsweep',
 'rootkit',
 'satan',
 'smurf',
 'spy',
 'teardrop',
 'warezclient',
 'warezmaster'}

In [11]:
# y_data is all the values under 'label' column 
# x_data is all columns except the last one.

y_data = x['label']
x_data = x[x.columns[:-1]]

noOfRows = len(x_data)
ntrain = (noOfRows * 2)//3

x_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.4, random_state=0)
print(len(x_train)," ",len(x_test))

296412   197608


In [13]:
le = preprocessing.LabelEncoder()
le.fit(x_train['protocol_type'])

x_train['protocol_type'] = le.transform(x_train['protocol_type'])
x_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
312915,0,0,ecr_i,SF,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00
322577,0,0,ecr_i,SF,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00
278229,0,0,ecr_i,SF,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00
458714,0,1,http,REJ,0,0,0,0,0,0,...,1,255,1.00,0.00,1.00,0.12,0.00,0.0,1.00,0.62
89473,0,1,http,SF,294,3614,0,0,0,0,...,3,255,1.00,0.00,0.33,0.03,0.00,0.0,0.00,0.00
299917,0,0,ecr_i,SF,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00
61137,0,1,private,S0,0,0,0,0,0,0,...,255,17,0.07,0.07,0.00,0.00,1.00,1.0,0.00,0.00
29596,0,1,http,SF,238,1951,0,0,0,0,...,18,255,1.00,0.00,0.06,0.01,0.00,0.0,0.00,0.00
350178,0,1,private,S0,0,0,0,0,0,0,...,255,19,0.07,0.08,0.00,0.00,1.00,1.0,0.00,0.00
431448,0,0,ecr_i,SF,520,0,0,0,0,0,...,255,255,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00


In [14]:
le.fit(x_train['service'])
x_train['service']=le.transform(x_train['service'])

le.fit(x_train['flag'])
x_train['flag']=le.transform(x_train['flag'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [0]:
le.fit(y_train)
y_train = le.transform(y_train)

In [16]:
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)